# Link to the Instruction Page
[Click here](https://chainladder-python.readthedocs.io/en/latest/tutorials/demo.html) to view the filled-in demo code.

# Setting Up
We will first need to install the package, as Google Colab's default environment doesn't have the chainladder package pre-installed. You will need to run this step using your terminal instead of using a python notebook when you are ready to install the package on your machine.

Simply execute `pip install chainladder`, Colab is smart enough to know that this is not a piece of python code, but to execute it in shell. FYI, `pip` stands for "Package Installer for Python".

Other commonly used packages, such as `numpy`, `pandas`, and `matplotlib` are already pre-installed, we just need to load them into our environment.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chainladder as cl

Let's check the version of chainladder that we have.

In [ ]:
print("chainladder:", _____)

# Your Journey Begins

Let's begin by looking at a sample dataset, called `clrd`, which is hosted on https://raw.githubusercontent.com/casact/chainladder-python/master/chainladder/utils/data/clrd.csv.

Let's load the dataset into the memory with `pandas`, then inspect it.

In [ ]:
clrd_df = pd.read_csv(_____)
clrd_df.head()

Can you list all of the unique accident years? How many are there?

In [ ]:
clrd_df[_____].unique()

In [ ]:
clrd_df[_____].nunique()

# Triangle Basics

Let's instantiate the `clrd_df` dataset and build a triangle object. Let's call it `clrd_tri`.

In [ ]:
clrd_tri = cl.Triangle(
    _____,
    origin=_____,
    development=_____,
    columns=["IncurLoss", "CumPaidLoss", "BulkLoss", "EarnedPremDIR", "EarnedPremCeded", "EarnedPremNet"],
    index = "LOB",
    cumulative=True,
)

clrd_tri

What does the paid triangle look like?

In [ ]:
clrd_tri[_____].sum()

How about incurred?

In [ ]:
clrd_tri[_____].sum()

The `clrd` dataset's `IncurLoss` actually includes IBNR, to get the case incurred, we would have to take out `BulkLoss`. Let's try to get the case incurred triangle.

In [ ]:
clrd_tri[_____].sum() - clrd_tri[_____].sum()

# Pandas-like Operations

Let's see how `.iloc[...]` and `.loc[...]` similarly to pandas. They take 4 parameters: [index, column, origin, valuation].

What if we want the row from AY 1998 `CumPaidLoss` data?

In [ ]:
clrd_tri.iloc[_____, _____, _____, _____].sum()

What if you only want the value at age 60?

In [ ]:
clrd_tri.iloc[_____, _____, _____, _____].sum()

Let's use `.loc[...]` to get the `CumPaidLoss` triangle.

In [ ]:
clrd_tri._____[_____, _____, _____, _____].sum()

What if you only want the row from 1992 accident year?

In [ ]:
clrd_tri._____[_____, _____, _____, _____].sum()

How do we get the latest `CumPaidLoss` diagonal only?

In [ ]:
clrd_tri["CumPaidLoss"].sum()._____

Very often, we want incremental triangles instead. Let's convert the Incurred triangle to the incremental form.

In [ ]:
clrd_tri["CumPaidLoss"].sum()._____

We can also convert the triangle to the valuation format, what we often see on Schedule Ps.

In [ ]:
clrd_tri["CumPaidLoss"].sum()._____

Another function that is often useful is the `.heatmap()` method. Let's inspect the incremental paid amount and see if there are trends.

In [ ]:
clrd_tri["CumPaidLoss"].sum()._____._____

Often, we need to get out of the chainladder Triangle object, and back to pandas, we can simply use `.to_frame()` to get the object back as a DataFrame.

In [ ]:
print(type(clrd_tri))
print(type(clrd_tri._____))

# Development

How can we get the `CumPaidLoss` link ratios?

In [ ]:
clrd_tri["CumPaidLoss"].sum()._____

We can also apply a `.heatmap()` to make it too, to help us visualize the highs and lows.

In [ ]:
clrd_tri["CumPaidLoss"].sum()._____._____

Let's get the LDFs for our paid triangle.

In [ ]:
cl.Development().fit(_____).ldf_

How about the CDFs?

In [ ]:
cl.Development().fit(_____)._____

We can also use only the latest 3 periods in the calculation of LDFs.

In [ ]:
cl.Development(n_periods=_____).fit(_____).ldf_

# Deterministic Models

Before we can build any models, we need to use `fit_transform()`, so that the object is actually modified with our the development pattern(s).

Set the development of the triangle to use only 3 periods.

In [ ]:
cl.Development(n_periods=_____)._____(_____)

Let's fit a chainladder model to our paid triangle.

In [ ]:
cl_mod = cl.Chainladder().fit(_____)
cl_mod

How can we get the model's ultimate estimate?

In [ ]:
cl_mod._____

How about just the IBNR?

In [ ]:
cl_mod._____

Let's fit an Expected Loss model, with an aprior of 90% on Premium, and get its ultimates.

In [ ]:
cl._____(apriori=_____).fit(
    _____, 
    sample_weight=_____
).ultimate_

Try it on the incurred triangle, which again, is `IncurLoss` - `BulkLoss`, do you get the same estimates?

In [ ]:
cl._____(apriori=0.90).fit(
    _____ - _____, 
    sample_weight=_____
).ultimate_

How about a Bornhuetter-Ferguson model?

In [ ]:
cl._____(apriori=_____).fit(
    _____, 
    sample_weight=_____
).ultimate_

How about Benktander, with 2 iterations?

In [ ]:
cl._____(apriori=_____, n_iters=_____).fit(
    _____, 
    sample_weight=_____
).ultimate_

How about Cape Cod?

In [ ]:
cl._____().fit(
    _____, 
    sample_weight=_____
).ultimate_

Let's store the Cape Cod model as `cc_result`. We can also use `.to_frame()` to leave `chainladder` and go to a `DataFrame`. Let's make a bar chart over origin years to see what they look like.

In [ ]:
cc_result = cl.CapeCod().fit(
    clrd_tri["CumPaidLoss"].sum(), sample_weight=clrd_tri["EarnedPremDIR"].sum().latest_diagonal
)

cc_result_ult = cc_result.ultimate_.to_frame()

plt.bar(
    _____, #x-axis, accident year
    _____, #y-axis, ultimates
)

# Pipelines

Pipeline is a tool that can streamline your workflow, it is a way to wrap multiple estimators into a single compact object. Let's see an example that includes a selection of development pattern and the chainladder model.

In [ ]:
pipe = cl.Pipeline(
    steps=[
    ('dev', cl.Development(average='simple')),
    ('model', cl.Chainladder())])

pipe

Once the pipeline is built, we can apply to any dataset we like, let's try it on the paid triangle and get their ultimate estimates.

In [ ]:
pipe.fit(_____).named_steps.model.ultimate_

You can use the same pipeline on other datasets as well, let's try it on the incurred triangle, which again, is `IncurLoss` - `BulkLoss`.

In [ ]:
pipe.fit(_____).named_steps.model.ultimate_

# Stochastic Models

The Mack's Chainladder model is available.

In [ ]:
mcl_mod = cl._____().fit(_____)
mcl_mod

There are many attributes that are available, such as `full_std_err_`, `total_process_risk_`, `total_parameter_risk_`, `mack_std_err_` and `total_mack_std_err_`.

In [ ]:
mcl_mod._____

MackChainladder also has a `summary_` attribute.

In [ ]:
mcl_mod._____

Let's make a graph, that shows the Paid and Unpaid as stacked bars, and error bars showing Mack Standard Errors.

In [ ]:
mcl_mod_df = mcl_mod.summary_.to_frame()

# bottom bar chart
plt.bar(
    _____, # category, year
    _____, # height, Latest (paid)
    label="Paid",
)

# top bar chart
plt.bar(
    _____, # category, year
    _____, # height, IBNR (unpaid)
    bottom=_____, # height, Latest (paid)
    yerr=_____, # Mack's standard error
    label="Unpaid",
)
plt.legend(loc="upper left")

ODP Bootstrap is also available. Let's bootstrap 10,000 paid triangles.

In [ ]:
clrd_tri_sampled = (
    cl._____(n_sims=_____).fit(_____).resampled_triangles_
)
clrd_tri_sampled

We can fit a basic chainladder to all sampled triangles. We now have 10,000 simulated chainladder models, all (most) with unique LDFs.

In [ ]:
cl_mod_bootstrapped = cl.Chainladder().fit(_____)
cl_mod_bootstrapped

We can use `predict()` to use the model characteristics (their unique LDFs) to predict our basic Incurred triangle. Let's inspect the prediction's means. 

In [ ]:
cl_mod_bootstrapped.predict(_____).ultimate_._____

Let's make another graph.

In [ ]:
plt.bar(
    _____, #x-axis, accident year
    _____, #y-axis, ultimates
    yerr=_____, #ultimates standard deviation
)